
# <!-- TITLE --> [GRISLI temperature] - Regression with a Dense Network (DNN)

<!-- DESC --> A Simple regression with a Dense Neural Network (DNN) 

## analysis of models (enventually stacking several models)

- AUTHOR : catritz 
- date : 3 january 2021
- new February 13 2021
- again April 2021

## version on relative vertical profile (varT)

## Objectives :
 - Predicts **temperature profile** from a set of parameters
 - with  **models calculated in DNN_with_varT.ipynb**


### Database : generated with read-GRISLI-files.ipynb

data in ./data/TGrisli_vert_AN40C008.csv and others

 - Thick: ice thickness (m)
 - phy: ghf (in mw/m2)
 - Ts:surface temperature (°C)
 - acc: accumulation rate (m/yr)
 - base: basal temperature (difference from the melting point)
 - ...
 
 **The vector of temperature profile**
 - T_01,T_02,T_03,T_04,T_05,T_06,T_07,T_08,T_09,T_10,T_11,T_12,T_13,T_14,T_15,T_16,T_17,T_18,T_19,T_20,T_21
 

 
### models (hdf5) and json files 

- in ./December-2020-hdf5-json.   
    * json files contain the dictionnary with all the parameters

### routines utilitaires in routines_cat_for_DNN_varT.py

- pretty(d, indent = 1):   # pretty print d'un dictionnaire
- nc2pd(file_in)           # read netcdf files and construct a dataframe
    * return pd_onefile
- select_some(df_tot,dic):   # select points according to criteria in dictionnary.
    * return data2
- diff_surf(df_in, dic):  # make the difference with surface temperature step
    * return df_out
- diff_model_data(ydiff, df_in)  * some calculation differences between predicted and target and merge with the initial dataframes
    * ydiff = ypred - y        where y is the initial data once  and ypred the one predicted by the mdoel
    * df_in is the initial dataframe (after select some)
    * return df_out
- get_norm(df,dic):    # calculate norms from df according to parameters
    * return x_norm


## Quelques routines Temperature 
- could be in a module as well

In [ ]:
def conductivity(T): # in W/m/K
    TT = 273.15
    Kcond = 9.828 * np.exp(-0.0057*(TT+T))
    # in the heat equation it will be nevessary to * by an = seconds/year
    return Kcond             
    
def heat_capacity(T): 
    TT = 273.15
    cg = 152.5 + 7.122 *(TT+T)   # in J/kg/K
    cg_rho = cg * 0.911 * 1.e3   # in J /m3/K
    
    return cg_rho    

def melting_T(z): # in °C
    rho_g = .911 * 1.e-2 * 9.807
    Tmelt = -0.0074 * rho_g * z
    return Tmelt       

In [ ]:
def T_profile(xprof,yprof,diff_tot_in,testrun):    # extract a vertical profile

    df_cp = diff_tot_in.copy(deep=True)  # to avoid modifying diff_tot_in
    
    # extract the column wanted by the value of x and y
    df_col = df_cp[(df_cp['x'] == xprof) & ( df_cp['y'] == yprof) & (df_cp["GRISLI_run"] == testrun)]

     #extract the thickness
    thick = df_col.H.values[0]
    z_depth = np.linspace(0.,thick, 21)   # create an array of depths
    

    #  list of columns for GRISLI temperature
    list_col = ['Ts','T_01', 'T_02', 'T_03', 'T_04',
           'T_05', 'T_06', 'T_07', 'T_08', 'T_09', 'T_10', 'T_11', 'T_12', 'T_13',
           'T_14', 'T_15', 'T_16', 'T_17', 'T_18', 'T_19', 'T_20']
    list_col2 = list_col[1:]

    # list of columns for DNN deviations
    list_colDNN = ['Ts', 'dT_01', 'dT_02', 'dT_03',
           'dT_04', 'dT_05', 'dT_06', 'dT_07', 'dT_08', 'dT_09', 'dT_10', 'dT_11',
           'dT_12', 'dT_13', 'dT_14', 'dT_15', 'dT_16', 'dT_17', 'dT_18', 'dT_19',
           'dT_20']
    # list of columns for DNN reconstructed
    list_DNN = ['T_00','T_01', 'T_02', 'T_03', 'T_04',
           'T_05', 'T_06', 'T_07', 'T_08', 'T_09', 'T_10', 'T_11', 'T_12', 'T_13',
           'T_14', 'T_15', 'T_16', 'T_17', 'T_18', 'T_19', 'T_20']
        
    list_colDNN2 = list_colDNN[1:]

    # extract the GRISLI temperature
    dd1 = df_col.copy(deep=True)  # if not, modify the original dataframe(warning)
    dd1[list_col2] = df_col[list_col2].add(df_col.Ts,axis=0)   # add the surface temperature"

   
    
    dd1bis = dd1[list_col]
    TGRISLI = np.ravel(dd1bis.values)
    
    # extract the DNN temperature

    dd2 = df_col.copy(deep=True)  # to avoid modifying the original dataframe(warning)
    dd2bis = dd2[list_colDNN2]
    dd2bis.columns = list_col2    # change the columns names  
    dDNN = dd1bis.add(dd2bis)     # add GRISLI temp and DNN result (they have the same columns names)  
    dDNN['T_00'] = dd1.Ts         # add the column TS as T_00
    dDNNfinal = dDNN[list_DNN]    #

    TDNN = np.ravel(dDNNfinal.values) # ravel change the shape from (1,21) to (21,)
    
    return z_depth, TGRISLI,TDNN 

## Import models

In [ ]:
import tensorflow as tf
from tensorflow import keras
#from keras.layers import Dropout
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm

import pandas as pd
import xarray as xr
import os,sys
import json
from datetime import date

sys.path.append('..')
import fidle.pwk as ooo

#ooo.init()
os.makedirs('./run/',   mode=0o750, exist_ok=True)

# my own modules that are in anaconca3/Programmes/Utils (in the python path)

import plotTprofile as pltp

import routines_cat_for_DNN_varT  as dnn

### Modify default plotting parameters and test on an example

In [ ]:
#plt.rcParams.keys()   # list setting of matplotlib

# %matplotlib inline
%matplotlib

# set the font globally


plt.rcParams.update({'font.family':'sans-serif'})
plt.rcParams.update({'text.color':'darkred'})    # for title
plt.rcParams.update({ 'font.size': 20})
plt.rcParams.update({ 'axes.linewidth': 2.})


x = [1, 2, 3, 4]
y = [234, 124,368, 343]

# Create the figure and axes objects
fig, ax = plt.subplots(1, figsize=(8, 6))
fig.suptitle('Example Of Plot With Grid Lines')

#ax.yaxis.tick_right()
#ax.yaxis.tick_left()

ax.set_xlabel('x-axis')
# ax.set_ylabel('y-axis', fontsize = 10)
ax.set_ylabel('y-axis')
plt.xticks()   #(fontsize = 15)

# Plot the data
ax.plot(x,y)

# Show the grid lines as dark grey lines
plt.grid(b=True, which='major', color='#666666', linestyle='-')

plt.show()

### input the data to model

In [ ]:
# all GRISLI runs

file08 = './data/Grisli_4_DNN_AN40C008.nc'
file09 = './data/Grisli_4_DNN_AN40C009.nc'
file10 = './data/Grisli_4_DNN_AN40C010.nc'
file11 = './data/Grisli_4_DNN_AN40C011.nc'

ncfile_list = [file08,file09,file10,file11]

# initialize DataFrame before loop
df_tot = pd.DataFrame()

# loop on the ncfiles
for file_in in ncfile_list:
    print (file_in)
    pd_onefile = dnn.nc2pdtd(file_in)  # time dependent version
    df_tot = df_tot.append(pd_onefile,ignore_index=True)  
#    print ('columns')
#    print(df_tot.columns)

## loop on the models

In [ ]:
df_tot.columns


In [ ]:
#dirmodel = './December-2020-hdf5-json/'
#dirmodel = './jan-fev-2021-hdf5-json/'
dirmodel = 'apr-2021-hdf5-json/'
#subtitle = 'december2020_test_'
#subtitle = 'january2021_test_'
subtitle = 'april2021_test_'
postitle = '_all_meanlog'

#ndeb = 7
#nend = 17
#ndeb = 6
#nend = 17
ndeb = 40
nend = 41

for i in range(ndeb,nend):
    
    # retrieve the json file from i
    numstring = "{:02n}".format(i)
    titi = dirmodel + subtitle+numstring+postitle  
    tutu = subtitle+numstring+postitle+'-weights'
    json_file = titi+'.json'
    print ('json_file: ', json_file)
    
    # retrieve the model from i 
    list_files = os.listdir(dirmodel)   # list of the dir with models  

    # eventually try syntax: [s for s in mylist if sub in s]
    
    for toto in list_files:   
        if tutu in toto:
            model_file = dirmodel + toto
    print('model_file: ', model_file)
   
    # read the json file
    with open(json_file) as jsonfile:
        dic_no = json.load(jsonfile)   # dic_no is the new dictionnary
        
        # eventuelly modify one of the conditions dic['user3']['prenom']
#        base_type = 'all'  
#        dic_no['select_data']['base'] = 'all'
#        print (dic_no['select_data'])
        
    # open the  model
    model = keras.models.load_model(model_file)

# prepare data

    # if the dataframe directly comes from nc2pd
    df_topredict = df_tot

    # remove floating part, thickness too small and eventually other caracteristics
    data_no = dnn.select_some(df_topredict,dic_no)
    
#    dd1 = data_no[(data_no['x'] == 0) & ( data_no['y'] == 0)]
#    dd2 = df_tot[(df_tot['x'] == 0) & ( df_tot['y'] == 0)]
                  
#    print ('data_no before difference', 
#           pd.Dataframe(dd1,columns = ['Ts','Tb','T00', 'T01'])
#    print("dd1",dd1.T_00)
#    print(dd1.Ts)
#    print("dd2",dd2.T_00)
#    print(dd2.Ts)

                 

    # to work in difference with surface temperature
           
    data_no = dnn.diff_surf(data_no,dic_no )
    
#    print ('data_no after difference', 
#           data_no[(data_no['x'] == 0) & ( data_no['y'] == 0)])
    


    # list of y columns
    list_yno = dic_no['y_columns']['list_y'] 
    y_no =  pd.DataFrame(data_no,columns=list_yno) 
    
    # normalize
    x_no = dnn.get_norm(data_no,dic_no)  

    # transform in numpy arrays
#    print (list_pred)
    x_no2 = np.array(x_no)
    y_no2 = np.array(y_no)


# run the model

    ypred_no2 = model.predict(x_no2)

# evaluate
    score = model.evaluate(x_no2, y_no2, verbose=0)

    print('x_test / mae       : {:5.4f}'.format(score[1]))
    print('x_test / mse       : {:5.4f}'.format(score[2]))
    
    # stack the results  in y_tot (numpy array)
    if i==ndeb:
        shape_tot = (nend-ndeb,ypred_no2.shape[0], ypred_no2.shape[1])         
        y_tot = np.zeros(shape_tot)
        print ('y_no2',y_no2.shape)
        print ('x_no2',x_no2.shape)

    y_tot[i-ndeb,:,:] = ypred_no2
    # end on the loop on the DNN models

    ymean_pred = y_tot.mean(axis=0)
    ystd_pred = np.std(y_tot,axis=0)

    #planstd = ystd_pred.mean(axis=1)  

In [ ]:
# make the difference predicted - initial
ydiff = ymean_pred - y_no2

# make some operation on the difference and add to the initial dataframe
diff_tot = dnn.diff_model_data(ydiff,data_no )
diff_tot.shape  
display(diff_tot.describe().style.format("{0:.2f}").set_caption("run 7-16"))  

In [ ]:
diff_tot.columns


### histograms

In [ ]:
### matplotlib

#%matplotlib notebook
#%matplotlib inline
%matplotlib

In [ ]:
# create a new figure window   plt.subplots()
#fig = plt.subplots(figsize=(20,15))
fig1 = plt.figure(figsize=(20,15))

# me_row
diffdata = diff_tot.me_row
#diffdata = diff_tot.dT_20
ax1 = fig1.add_subplot(221)
#ax1 = plt.axes()

#List_bins = [-5,-2,-1,-0.5,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.5,1,2,5]
#List_bins = [-5,-2,-1,-0.5,0,0.5,1,2,5]
lnp = np.arange(-20,21,1)/10
List_bins = lnp.tolist()


hist, bins = np.histogram (diffdata, bins = List_bins) 
#print (hist) 

plt.hist(diffdata, bins = List_bins) 
plt.title("DNN predicted - GRISLI (deg)\n averaged over each ice column")
#

diffdata = diff_tot.sigma_row

ax2 = fig1.add_subplot(222)

lnp = np.arange(0,41,1)/20
List_bins = lnp.tolist()

hist, bins = np.histogram (diffdata, bins = List_bins) 

#print (hist) 
plt.hist(diffdata, bins = List_bins) 

plt.title("histogram of sigma (deg) \n calculated for each ice column") 


namefig = 'figure1.png'
plt.savefig(namefig)

plt.show()

In [ ]:
fig2 = plt.figure(figsize=(20,15))

# figure left
#diffdata = (diff_tot.dT_10 - diff_tot.dT_01) / diff_tot.H *20*1000 

diffdata = diff_tot.dT_10 
ax1 = fig2.add_subplot(221)

#List_bins = [-5,-2,-1,-0.5,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.5,1,2,5]
#List_bins = [-5,-2,-1,-0.5,0,0.5,1,2,5]
lnp = np.arange(-20,21,1)/10   # pour T
#lnp = np.arange(-200,210,1)/10   # pour dTdz

List_bins = lnp.tolist()


hist, bins = np.histogram (diffdata, bins = List_bins) 
#print (hist) 

plt.hist(diffdata, bins = List_bins) 
plt.title("DNN predicted - GRISLI (deg) \n midle of the ice columns")           

# bottom temperature

diffdata = diff_tot.dT_20
#diffdata = (diff_tot.T_10 - diff_tot.T_01) / diff_tot.H *20 *1000

ax2 = fig2.add_subplot(222)

#lnp = np.arange(0,41,1)/20
#List_bins = lnp.tolist()
lnp = np.arange(-20,21,1)/10   # pour T
#lnp = np.arange(0,400,1)/1

List_bins = lnp.tolist()

hist, bins = np.histogram (diffdata, bins = List_bins) 

#print (hist) 
plt.hist(diffdata, bins = List_bins) 


plt.title("DNN predicted - GRISLI (deg) \n ice bottom")



namefig = 'figure2-dec2020-7-16.png'
plt.savefig(namefig)
plt.show()

In [ ]:

diff_tot.columns

## Plot a map

In [ ]:
# select the grisli run
grisliname = file09

s = '/'
pos = grisliname.rfind(s)
posend = grisliname.rfind('.nc')
testrun = grisliname[pos+1:posend]
print(testrun)

df_cp = diff_tot.copy(deep=True)
data2 = df_cp[df_cp["GRISLI_run"] == testrun]
print (data2.shape)
print (diff_tot.shape)

#data2.columns

# get the x y coordinates
xcoord = data2[data2.columns[0]]
ycoord = data2[data2.columns[1]]

xcoord2 = np.array(xcoord)/1000.
ycoord2 = np.array(ycoord)/1000.
xcoord2.shape

# axes

fig3 = plt.figure(figsize=(15,8))

titlemap = 'difference Emulator - GRISLI (deg) \n vertically integrated'
#titlemap = 'difference Emulator - GRISLI (deg) \n for basal temperature'
z = data2.me_row.values
#z = data2.dT_20.values

ax1 = fig3.add_subplot(121)

ax1.set(title = titlemap , xlabel="x (km)", ylabel ="y(km)")
ax1.set(xlim = (-2800,2800), ylim = (-2800,2800))
ax1.set(aspect = 'equal')
#ax1.set (facecolor = 'lightgrey')
ax1.set (facecolor = (0.95,0.95,0.95))


plt.scatter(xcoord2,ycoord2,marker='o',c=z,s=10,
#            cmap='rainbow')

#             cmap='rainbow', norm=LogNorm(), vmin = 1.e-5,vmax = 1.e-1)
#             cmap='rainbow', norm=LogNorm(), vmin = 0.1, vmax = 2000)
            cmap='RdYlBu_r', vmin=-2., vmax =2.)

plt.colorbar(fraction=0.046, pad=0.08, label = '(deg)')
#plt.colorbar() # draw colorbar
namefig = 'figure1-feb15-2021.png'
plt.savefig(namefig)

plt.show

In [ ]:
diff_tot.columns

## Plot en fonction des inputs

In [ ]:
#import mpl_toolkits.axisartist as AA

fig = plt.figure(figsize=(20,15))


# petit bloc pour changer les variables qu'on veut


#diffdata = diff_tot[diff_tot.Tb < 0] 

diffdata = diff_tot

title_fig="Test Samples"
title_fig="data_no"

#xlabl = "diff me  (deg)"
xlabl = "diff basal temperature"
#Axes.set_frame_on
xticks = (-4,-3,-2,-1,0,1,2,3,4)
#xval = diffdata.dT_20.values
#xval = diffdata.me_row.values
xval = diffdata.dT_20.values


ax = plt.subplot(2, 2, 1)   # carre de 2 par 2, premier element. Les deux formulations sont équivalents
#ax = fig.add_subplot(221, )

ax.set(title=title_fig, 
       xlabel= xlabl, 
       ylabel ="Velocity (m/yr)",
       xlim = (-4,4),
       ylim = (0,200),
       xticks=xticks)

plt.grid(b=True, which='major', color='#666666', linestyle='-')

ax.grid(True,zorder=10)



plt.scatter(x=xval,y=diffdata.Us.values,
            marker='o', s=5,
            c=diffdata.Tb.values,
            cmap='rainbow')

plt.colorbar(label='Tb (°C)') # draw colorbar
#----------------------------------------------------

ax2 = fig.add_subplot(222)

ax2.set(title="", 
       xlabel=xlabl, 
       ylabel ="Thickness (m)",
       xlim = (-4,4),
       ylim = (1000,4500),
       xticks= xticks)

ax2.grid(True,zorder=10)

plt.scatter(x=xval,y=diffdata.H.values,
            marker='o', s=5,
            c=diffdata.Tb.values,
            cmap='rainbow')
plt.colorbar(label='Tb') # draw colorbar
#----------------------------------------------------
ax3 = fig.add_subplot(223)

ax3.set(title="", 
       xlabel=xlabl, 
       ylabel ="ghf (mW/m2)",
       xlim = (-4,4),
       ylim = (30,120),
       xticks=xticks)

ax3.grid(True,zorder=10)

plt.scatter(x=xval,y=diffdata.ghf.values,
            marker='o', s=5,
            c=diffdata.Tb.values,
            cmap='rainbow')
plt.colorbar(label='Tb') # draw colorbar
#----------------------------------------------------
ax4 = fig.add_subplot(224)


ax4.set(title="", 
       xlabel=xlabl, 
       ylabel ="Bminmax (m)",
       xlim = (-4,4),
       ylim = (0,2000),
       xticks=xticks)


ax4.set(title="", 
       xlabel=xlabl, 
       ylabel ="Bm (m/yr)",
       xlim = (-4,4),
       ylim = (0,0.5),
       xticks=xticks)

ax4.grid(True,zorder=10)

#plt.scatter(x=xval,y=diffdata.Bminmax.values,
plt.scatter(x=xval,y=diffdata.Bm.values,
            marker='o', s=5,
            c=diffdata.Tb.values,
            cmap='rainbow')

#           cmap='rainbow', norm=LogNorm(), vmin = 0.001, vmax = 0.1) 

plt.colorbar(label='Tb') # draw colorbar
plt.show
#namefig='figure-'+title+'.png'
#plt.savefig(namefig)   # save the figure in the file namefig

In [ ]:
df_bad = diff_tot[(diff_tot['sigma_row'] > 1) & (diff_tot["GRISLI_run"] == testrun)]
listvar = ['x','y','H','dT_10','dT_20']
df_bad[listvar]

### Plot  Profile

In [ ]:
# select oints that do not work


xprof = -1120000 
yprof = -920000



zz,xx1,xx2 =  T_profile(xprof,yprof,diff_tot,testrun)  # return depth, TGRISLI, TDNN

### Plot Profile

fig = plt.figure(figsize=(20,15))

title_fig="Vertical T profiles"


xlabl = "temperature (°C)"
ylabl = "Depth (m)"

ax = fig.add_subplot(121)

ax.set(title=title_fig, 
       xlabel= xlabl, 
       ylabel =ylabl,
       xlim = (-60,0),
       ylim = (3500,0),
#       xticks=xticks
      )

# plot the melting point
zbot= 3500
Tbot = melting_T(zbot)
z_melt = np.array([0,zbot])
T_melt = np.array([0,Tbot])
print (Tbot)

col='r'

ax.plot(T_melt, z_melt, color = 'darkorange', linestyle='-.', 
        label= str('melting T'), linewidth = 4 )
ax.plot(xx1, zz, color = col, label= str('GRISLI'), linewidth = 4 )
ax.plot(xx2, zz, color = 'b',linestyle='--', label= str('DNN'), linewidth = 4 )

ax.legend(loc=3)

# another point 


zz,xx1,xx2 =  T_profile(xprof,yprof,diff_tot,testrun)  # return depth, TGRISLI, TDNN

ax.plot(xx1, zz, color = col, linewidth = 4 )
ax.plot(xx2, zz, color = 'b',linestyle='--',linewidth = 4 )

xprof = 1040000.0
yprof = 720000.0


zz,xx1,xx2 =  T_profile(xprof,yprof,diff_tot,testrun)  # return depth, TGRISLI, TDNN

ax.plot(xx1, zz, color = col,linewidth = 4)
ax.plot(xx2, zz, color = 'b',linestyle='--', linewidth = 4 )

xprof = 1280000.0
yprof = 480000.0


zz,xx1,xx2 =  T_profile(xprof,yprof,diff_tot,testrun)  # return depth, TGRISLI, TDNN

ax.plot(xx1, zz, color = col,linewidth = 4)
ax.plot(xx2, zz, color = 'b',linestyle='--', linewidth = 4 )

xprof = 1280000.0
yprof = -1360000.0 


zz,xx1,xx2 =  T_profile(xprof,yprof,diff_tot,testrun)  # return depth, TGRISLI, TDNN

ax.plot(xx1, zz, color = col,linewidth = 4)
ax.plot(xx2, zz, color = 'b',linestyle='--', linewidth = 4 )

xprof = 1680000.0
yprof = -1200000.0 

zz,xx1,xx2 =  T_profile(xprof,yprof,diff_tot,testrun)  # return depth, TGRISLI, TDNN

ax.plot(xx1, zz, color = col,linewidth = 4)
ax.plot(xx2, zz, color = 'b',linestyle='--', linewidth = 4 )


plt.grid(b=True, which='major', color='#666666', linestyle='-')


plt.show()

In [ ]:
# select one point


xprof = 960000
yprof = 240000

xprof = 1120000
yprof = 200000

zz,xx1,xx2 =  T_profile(xprof,yprof,diff_tot,testrun)  # return depth, TGRISLI, TDNN

### Plot Profile

fig = plt.figure(figsize=(20,15))

title_fig="Vertical T profiles"


xlabl = "temperature (°C)"
ylabl = "Depth (m)"

ax = fig.add_subplot(121)

ax.set(title=title_fig, 
       xlabel= xlabl, 
       ylabel =ylabl,
       xlim = (-60,0),
       ylim = (3500,0),
#       xticks=xticks
      )

# plot the melting point
zbot= 3500
Tbot = melting_T(zbot)
z_melt = np.array([0,zbot])
T_melt = np.array([0,Tbot])
print (Tbot)

col='r'

ax.plot(T_melt, z_melt, color = 'darkorange', linestyle='-.', 
        label= str('melting T'), linewidth = 4 )
ax.plot(xx1, zz, color = col, label= str('GRISLI'), linewidth = 4 )
ax.plot(xx2, zz, color = 'b',linestyle='--', label= str('DNN'), linewidth = 4 )

ax.legend(loc=3)

# another point 


zz,xx1,xx2 =  T_profile(xprof,yprof,diff_tot,testrun)  # return depth, TGRISLI, TDNN

ax.plot(xx1, zz, color = col, linewidth = 4 )
ax.plot(xx2, zz, color = 'b',linestyle='--',linewidth = 4 )

xprof = - 1040000
yprof = -360000 

zz,xx1,xx2 =  T_profile(xprof,yprof,diff_tot,testrun)  # return depth, TGRISLI, TDNN

ax.plot(xx1, zz, color = col,linewidth = 4)
ax.plot(xx2, zz, color = 'b',linestyle='--', linewidth = 4 )

xprof = - 600000
yprof = -720000 

zz,xx1,xx2 =  T_profile(xprof,yprof,diff_tot,testrun)  # return depth, TGRISLI, TDNN

ax.plot(xx1, zz, color = col,linewidth = 4)
ax.plot(xx2, zz, color = 'b',linestyle='--', linewidth = 4 )

xprof =  1600000
yprof = -1040000 

zz,xx1,xx2 =  T_profile(xprof,yprof,diff_tot,testrun)  # return depth, TGRISLI, TDNN

ax.plot(xx1, zz, color = col,linewidth = 4)
ax.plot(xx2, zz, color = 'b',linestyle='--', linewidth = 4 )

xprof =  1720000
yprof =  120000 

zz,xx1,xx2 =  T_profile(xprof,yprof,diff_tot,testrun)  # return depth, TGRISLI, TDNN

ax.plot(xx1, zz, color = col,linewidth = 4)
ax.plot(xx2, zz, color = 'b',linestyle='--', linewidth = 4 )


plt.grid(b=True, which='major', color='#666666', linestyle='-')


plt.show()

In [ ]:
"""
df_cp = diff_tot.copy(deep=True)


df_col = df_cp[(df_cp['x'] == xprof) & ( df_cp['y'] == yprof) & (df_cp["GRISLI_run"] == testrun)]

#df_col.shape
df_col.columns

#extract the thickness
Thick = df_col.H.values[0]

### reconstruct the temperature profiles


# the DNN is trained on T-Ts 
# T_?? = TGrisli - Ts 
# dT_?? is the DNN of T_?? 
# T_DNN = dT_?? + Ts 


# list of GRISLI temperature
list_col = ['Ts','T_01', 'T_02', 'T_03', 'T_04',
	   'T_05', 'T_06', 'T_07', 'T_08', 'T_09', 'T_10', 'T_11', 'T_12', 'T_13',
	   'T_14', 'T_15', 'T_16', 'T_17', 'T_18', 'T_19', 'T_20']
list_col2 = list_col[1:]

# list of DNN deviations
list_colDNN = ['Ts', 'dT_01', 'dT_02', 'dT_03',
	   'dT_04', 'dT_05', 'dT_06', 'dT_07', 'dT_08', 'dT_09', 'dT_10', 'dT_11',
	   'dT_12', 'dT_13', 'dT_14', 'dT_15', 'dT_16', 'dT_17', 'dT_18', 'dT_19',
	   'dT_20']
list_colDNN2 = list_colDNN[1:]



#print(list_col2)

# extract the initial temperature
dd1 = df_col.copy(deep=True)  # if not, modify the original dataframe(warning)
dd1[list_col2] = df_col[list_col2].add(df_col.Ts,axis=0)   # add the surface temperature"
print('Ts =', df_col.Ts)

dd1bis = dd1[list_col2]
print("dd1bis")
print(dd1bis.values)
dd2[list_colDNN]
# extract the DNN temperature

#print("dd2")
dd2 = df_col.copy(deep=True)  # if not, modify the original dataframe(warning)
#print(dd2[list_colDNN])
dd2bis = dd2[list_colDNN2]


#print("dd2bis")
#print(dd2bis)

dd2bis.columns = list_col2
#print("dd2bis")
#print(dd2bis)

# sum dd1bis and dd2bis
dDNN = dd1bis.add(dd2bis)
#print('--------------------------')


# add the column TS as T_00
dDNN['T_00'] = dd1.Ts
#print("dDNN")
#print(dDDN)

# change the position of T_00
list_DNN = ['T_00','T_01', 'T_02', 'T_03', 'T_04',
	   'T_05', 'T_06', 'T_07', 'T_08', 'T_09', 'T_10', 'T_11', 'T_12', 'T_13',
	   'T_14', 'T_15', 'T_16', 'T_17', 'T_18', 'T_19', 'T_20']
#print(list_DNN)

dDNNfinal = dDNN[list_DNN]

#print('--------------------------')
#print("dDDN final")
#print(dDNNfinal)


print (dDNNfinal.values)
"""

## write netcdf file with the output

In [ ]:
# mesh of x and y
X, Y = np.meshgrid(xcoord, ycoord, copy=False)

In [ ]:
#index_test = data2.index
data.index.shape

Xmin = data.X.min()
Ymin = data.Y.min()
Xmax = data.X.max()
Ymax = data.Y.max()

dx = data.X[1]-data.X[0]

nx = int(round((Xmax-Xmin)/dx +1))
ny = int(round((Ymax-Ymin)/dx +1))
nz = y_test2.shape[1]

# indexes i,j of a line given its inded in the dataframe

T3D_data = np.zeros((nz,ny,nx))
T3D_dnn  = np.zeros((nz,ny,nx))

In [ ]:
# points where the DNN is applied
xx = xcoord.values   # xcoordinate of the point
yy = ycoord.values   # y coordinate of the point
y_pred.shape   # DNN result
y_test2.shape  # from GRISLI

# number of points
np = xcoord.shape[0]
#print(np)

#T3D_data = 0.
#T3D_dnn  = 0.

for column in range(np):
    i = int(round(xx[column] - Xmin) /dx)
    j = int(round(yy[column] - Ymin) /dx)
#    print(column,i,j)

    T3D_data[:,j,i] = y_test2[column,:]
    T3D_dnn[:,j,i]  = y_pred[column,:]
    
deltaT =  T3D_dnn - T3D_data

In [ ]:
# transform to dataArray


T1 = xr.DataArray(
    data=T3D_data,                     # data from numpy array
    dims=("z","x","y"),                   # dims copied from initial file
    name="Tgrisli"
    )

T2 = xr.DataArray(
    data=T3D_dnn,                     # data from numpy array
    dims=("z","x","y"),                   # dims copied from initial file
    name="Tdnn"
    )
dT = xr.DataArray(
    data=T3D_dnn-T3D_data,                     # data from numpy array
    dims=("z","x","y"),                   # dims copied from initial file
    name="DiffTdnn"
    )

dT.var

In [ ]:
T1.to_netcdf('test-T3D_grisli.nc')
T2.to_netcdf('test-T3D_dnn.nc')
dT.to_netcdf('diffT-dnn-grisli.nc')

In [ ]:
%matplotlib notebook
buffer.plot(cmap='rainbow', vmin = -2, vmax = 2)
plt.show()